# Amazon Shoe Department Chat-Bot
In this notebook, we will apply a RAG system to the same problem mentioned in the first session. A RAG system is designed to answer questions about shoes that are available in store. The data used here is downloaded from [here](https://data.world/crawlfeeds/amazon-uk-shoes-dataset).

This notebook was prepared using some of the material from these two sources:
* [Simple RAG for GitHub issues using Hugging Face Zephyr and LangChain](https://colab.research.google.com/github/huggingface/cookbook/blob/main/notebooks/en/rag_zephyr_langchain.ipynb#scrollTo=Kih21u1tyr-I)
* [Advanced RAG on Hugging Face documentation using LangChain](https://colab.research.google.com/github/huggingface/cookbook/blob/main/notebooks/en/advanced_rag.ipynb#scrollTo=VjVqmDGh9-9N)

In [ ]:
!pip install -q torch transformers transformers accelerate bitsandbytes langchain langchain-core sentence-transformers openpyxl pacmap datasets langchain-community ragatouille

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Downloading Data
All the data required for this session are downloaded here.

In [ ]:
!gdown https://drive.google.com/uc?id=1ZK73jV_3pGbs42gNIo0BYdtpfTcBrTfM

In [ ]:
!unzip -q /content/data.zip
!unzip -q /content/data/images.zip

## Loading Product Data
We parsed the product data into textual data that can be passed to LLMs.

In [ ]:
# Loading product data (Show dataset and data preparation)
path = "/content/data/amazon_shoe_database.jsonl"

# Load documents
data = []


In [ ]:
data[0]

In [ ]:
print(data[0]["text"])

In [ ]:
# If you plan to use the LangChain utilities, you can convert the JSON data to
# LangChain Documents to integrate with the LangChain library
from langchain_core.documents import Document

docs = []
for item in data:
  text = item["text"]
  metadata = {key: item[key] for key in ["url", "closure", "price", "brand", "department"]}
  doc = Document.construct(page_content=text, id=item["id"], metadata=metadata)
  docs.append(doc)

## Embedding model
Here we load the embedding model that we will use to create embeddings for our documents. We will be using [Jina AI's](https://jina.ai/) CLIP V1 model, available on [HuggingFace](https://huggingface.co/jinaai/jina-clip-v1).

The embeddings have already been computed earlier and stored in `/content/data/jinaai_doc_embeddings.jsonl`.

In [ ]:
!pip install -q einops timm pillow

In [ ]:
from transformers import AutoModel

# Initialize the model
embedding_model_id = 'jinaai/jina-clip-v1'
embedding_model = AutoModel.from_pretrained(embedding_model_id, trust_remote_code=True)
embedding_model = embedding_model.to('cuda') # Moving to GPU

In [ ]:
# Creating embeddings for our documents
# from tqdm import tqdm
# save_path = "doc_embeddings_save_path.jsonl"

# # Read existing doc embeddings
# if os.path.exists(save_path):
#   with open(save_path, 'r') as f:
#     existing_docs = [json.loads(line) for line in f]
#   existing_product_ids = set([doc["id"] for doc in existing_docs])
# else:
#   existing_product_ids = set()

# doc_embeddings = []
# for doc in tqdm(data):

#   product_id = doc["id"]

#   if product_id in existing_product_ids:
#     continue

#   embedding = model.encode_text(doc["text"])

#   result = {
#       "product_id": product_id,
#       "embedding": embedding.tolist()
#   }
#   doc_embeddings.append(result)

#   with open(save_path, 'a') as f:
#     f.write(json.dumps(result) + '\n')

In [ ]:
# Loading document embeddings
doc_embeddings = {}
path = "/content/data/jinaai_doc_embeddings.jsonl"
with open(path, "r") as f:
    for line in f:
        embedding_info = json.loads(line)
        item_id = embedding_info["product_id"]
        embedding = embedding_info["embedding"]
        doc_embeddings[item_id] = embedding

## Vector Database
We will directly use the Chroma DB without relying on the LangChain wrapper.

In [ ]:
# Installing ChromaDB library
!pip install -q chromadb

We will create a database using our precomputed embeddings.

In [ ]:
import chromadb

# Initializing client
chroma_client = chromadb.Client()

# Creating collection
doc_collection = chroma_client.create_collection(
    name="doc_database",
    metadata={"hnsw:space": "cosine"} # The similarity measure used to get query neighbours
    ) # You can define an embedding function here

In [ ]:
document_ids = [doc["id"] for doc in data]
documents = [doc["text"] for doc in data]
metadatas = []

for doc in data:
  metadata = {key: doc[key] for key in ["url", "closure", "price", "brand", "department"]}
  metadatas.append(metadata)

embeddings = [doc_embeddings[doc_id] for doc_id in document_ids]

doc_collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=document_ids
)

When retrieving a document from the database, the matching will be made based on the query embedding and the embeddings in the database. The top K documents will be returned with their metadata. Using the metadata, we can provide additional information to the user, such as URLs to the products that the model mentioned.

In [ ]:
query = "Men sneakers from Reebok"
query_embedding = embedding_model.encode_text(query)
relevant_docs = doc_collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=2 # Number of documents to return
)
relevant_docs["metadatas"]

## Generative Model
We will use the same Llama 3 - 8B - Instruct that you saw earlier. However, we will now use it without finetuning and rely on the external database to provide the LLM with the required information about the products.

In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

llm = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

In [ ]:
# Testing the model on our domain
query = "What are the best Geox shoes for men?"
out = llm(query)
print(out[0]["generated_text"])

The best practice is to use the prompt format specific to the model in use. Llama 3 prompt format can be found [here](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/).

In [ ]:
# Create prompt template based on meta template

def get_llama_prompt(query, context):
  pass

Now we will create our own pipeline. LangChain allows for creating pipelines using few code lines. However, if you want to add specific features to your system, it may be more flexible to create your own pipeline.

In [ ]:
# Create RAG pipeline
class RAG:
    pass

In [ ]:
rag_pipeline = RAG(
    database=doc_collection,
    embedding_model=embedding_model,
    model_pipeline=llm,
    prompt_template_func=get_llama_prompt,
)

In [ ]:
# Testing the pipeline
query = "Suggest me some Geox shoes for men and give me their prices"
result = rag_pipeline.invoke(query)

print(f"Answer: {result['answer']}")

In [ ]:
# Did the model use our data or its internal data?
print("\n\n".join(result["docs"]))

Now we will add an extra feature to the chat-bot. We will load the images of the products that are relevant to the user query and show them to the user.

In [ ]:
import os
from PIL import Image

def image_grid(imgs, rows, cols):
    """ Function to show images"""
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
# Add load images capability to the pipeline
class RAG:
    def __init__(
          self,
          database,
          embedding_model,
          model_pipeline,
          prompt_template_func,
          n_docs=3
          ):
        self.database = database
        self.embedding_model = embedding_model
        self.model_pipeline = model_pipeline
        self.get_prompt = prompt_template_func
        self.n_docs = n_docs
        images_dir = "/content/images/images"
        self.id2image = {
            file_name.split(".")[0]: os.path.join(images_dir, file_name) for file_name in os.listdir(images_dir)
        }

    def get_context(self, docs):
        return "\n\n".join([doc for doc in docs])

    def load_images(self, id_list):
        image_paths = [self.id2image[product_id] for product_id in id_list if product_id in self.id2image]
        images = [Image.open(image_path) for image_path in image_paths]
        return images

    def invoke(self, query):
        query_embedding = self.embedding_model.encode_text(query)
        relevant_data = self.database.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=self.n_docs
        )
        docs = relevant_data["documents"][0]
        ids = relevant_data["ids"][0]
        context = self.get_context(docs)
        prompt = self.get_prompt(query=query, context=context)
        answer = self.model_pipeline(prompt)[0]['generated_text']
        images = self.load_images(ids)

        # Keep one image for each product
        return {'docs': docs, 'prompt': prompt, 'answer': answer, "images": images}

In [ ]:
rag_pipeline = RAG(
    database=doc_collection,
    embedding_model=embedding_model,
    model_pipeline=llm,
    prompt_template_func=get_llama_prompt,
)

In [ ]:
query = "Suggest me some Reebok shoes for men and give me their prices"
result = rag_pipeline.invoke(query)

print(f"Answer: {result['answer']}")
if result["images"]:
  display(image_grid(result["images"], 1, len(result["images"])))

## Using image embeddings
The reason that we used the [Jina AI](https://jina.ai/) CLIP model is that it is a multimodal (text + image) embedding model. A multimodal embedding model can produce embeddings for images and text that are semantically comparable, i.e., an embedding of an image of a cat will be similar to the embedding of the word "cat".

In [ ]:
from PIL import Image

# Loading example images
dir_path = "/content/data/example_images/animal_images/"
image_files = os.listdir(dir_path)
images = [Image.open(os.path.join(dir_path, file)) for file in image_files]

# Displaying example images
fig, axs = plt.subplots(1, len(images), figsize=(15, 5))
for i, image in enumerate(images):
    axs[i].imshow(image)
    axs[i].axis('off')
plt.show()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query = "An image of a cat in a grass field"

query_embedding = embedding_model.encode_text(query)
image_embeddings = []
similarities = []
for image in images:
    image_embedding = embedding_model.encode_image(image)
    image_embeddings.append(image_embedding)
    similarity = cosine_similarity([query_embedding], [image_embedding])[0][0]
    similarities.append(similarity)

similarities = np.array(similarities)
order_by_similarity = similarities.argsort()[::-1]
ordered_images = [images[i] for i in order_by_similarity][:3]

fig, axs = plt.subplots(1, len(ordered_images), figsize=(15, 5))
for i, image in enumerate(ordered_images):
    axs[i].imshow(image)
    axs[i].axis('off')

plt.show()

We already computed the embeddings for the images earlier. We load them here:

In [ ]:
img_embeddings = {}
path = "/content/data/jinaai_image_embeddings.jsonl"
with open(path, "r") as f:
    for line in f:
        embedding_info = json.loads(line)
        item_id = embedding_info["product_id"]
        embedding = embedding_info["embedding"]
        img_embeddings[item_id] = embedding

We will now create a database using the embeddings of the images:

In [ ]:
import chromadb

# Initializing client
chroma_client = chromadb.Client()

# If a collection already exists you must delete it
# chroma_client.delete_collection("img_database")

# Creating collection
img_collection = chroma_client.create_collection(
    name="img_database",
    metadata={"hnsw:space": "cosine"} # The similarity measure used to get query neighbours
    ) # You can define an embedding function here

In [ ]:
document_ids = [product_id for product_id in img_embeddings.keys()]
docs_by_id = {doc["id"]: doc for doc in data}
documents = [docs_by_id[doc_id]["text"] for doc_id in document_ids]
metadatas = []

for doc_id in document_ids:
  doc = docs_by_id[doc_id]
  metadata = {key: doc[key] for key in ["url", "closure", "price", "brand", "department"]}
  metadatas.append(metadata)

embeddings = [img_embeddings[doc_id] for doc_id in document_ids]

img_collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=document_ids
)

Now we can query this database by embedding the user query and searching for similar image embeddings in the database. This only works with multimodal models that can project images and text to the same latent space.

In [ ]:
# A dictionary that maps each product ID to its image path
images_dir = "/content/images/images/"
id2image = {
    file_name.split(".")[0]: os.path.join(images_dir, file_name) for file_name in os.listdir(images_dir)
}

In [ ]:
query = "I want a yellow and blue sneaker"

query_embedding = embedding_model.encode_text(query)
relevant_docs = img_collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=2 # Number of documents to return
)

product_ids = relevant_docs["ids"][0]
image_paths = [id2image[product_id] for product_id in product_ids]
images = [Image.open(product_image) for product_image in image_paths]

image_grid(images, 1, len(images))

We can also search for products that look like a certain image:

In [ ]:
# Searching by image
search_image_path = "/content/data/example_images/shoe_images/banana_shoes.jpeg"
# search_image_path = "/content/data/example_images/shoe_images/clown_shoes.jpg"
search_image = Image.open(search_image_path)
search_image


In [ ]:
search_image_embedding = embedding_model.encode_image(search_image)
relevant_docs = img_collection.query(
    query_embeddings=[search_image_embedding.tolist()],
    n_results=2 # Number of documents to return
)

product_ids = relevant_docs["ids"][0]

image_paths = [id2image[product_id] for product_id in product_ids]
images = [Image.open(product_image) for product_image in image_paths]

image_grid(images, 1, len(images))

## Improving the RAG system

### Evaluation standard
In order to be able to improve the RAG system, we first need to know how to evaluate its performance. Evaluation of LLMs on downstream tasks is an extremely hard task. The reason is that unlike classification or regrassion, the output is not structured (text) and there is no single true "label". I strongly suggest listening to [this talk](https://www.youtube.com/watch?v=2CIIQ5KZWUM) to learn about challenges in evaluating LLMs on downstream tasks.

### Filtering metadata
We can narrow down our search by filtering based some fields in the metadata, such as search products in a certain price range or from a certain brand. This is a trivial feature in all databases; however, integrating it with a conversational bot is not trivial. One way would be to have an agent, i.e., an LLM, to extract information from the user query and use it to filter the search.

In [ ]:
query = "Men sneakers from Reebok"
query_embedding = embedding_model.encode_text(query)
relevant_docs = doc_collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=2, # Number of documents to return
    where={"department": "men"}
)
relevant_docs["metadatas"]

### Reranking documents
The vector databases use approximate approaches to ensure fast retrieval. A common approach is to retrieve more documents than required and then rerank them using a better retrieval approach. [Colbertv2](https://arxiv.org/abs/2112.01488) is a good choice for this purpose. It can be easily used through [the RAGatouille library](https://github.com/bclavie/RAGatouille).

### Finetuning
A recent paper [RAFT: Adapting Language Model to Domain Specific RAG](https://arxiv.org/abs/2403.10131) showed improved performance by finetuning an LLM on answering questions given documents. Specifically, they show noticable improvement in performance when using chain-of-thought reasoning when finetuning. Read more about chain-of-thought reasoning [here](https://www.promptingguide.ai/techniques/cot).